In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-dataset/Data_Patient.csv


In [2]:
import pandas as pd
df = pd.read_csv('/kaggle/input/heart-dataset/Data_Patient.csv')
df.head()

,id,Age,Gender,Type,Blood_pressure,Cholesterol,Heartbeat,Thalassemia,Result
0,Patient_01,63,Male,Typical angina,145,233,150,6.0,0
1,Patient_02,67,Male,Asymptomatic,160,286,108,3.0,1
2,Patient_03,67,Male,Asymptomatic,120,229,129,7.0,1
3,Patient_04,37,Male,Non-anginal pain,130,250,187,3.0,0
4,Patient_05,41,Female,Atypical angina,130,204,172,NaN,0


In [3]:
import pandas_profiling

In [4]:
# EDA

pandas_profiling.ProfileReport(df)

Summarize dataset:   0%|          | 0/22 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              300 non-null    object 
 1   Age             300 non-null    int64  
 2   Gender          300 non-null    object 
 3   Type            295 non-null    object 
 4   Blood_pressure  300 non-null    int64  
 5   Cholesterol     300 non-null    int64  
 6   Heartbeat       300 non-null    int64  
 7   Thalassemia     293 non-null    float64
 8   Result          300 non-null    int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 21.2+ KB


In [6]:
df.id = df.id.astype('category')

In [7]:
df.Gender = df.Gender.astype('category')

In [8]:
df.Type.nunique()

4

In [9]:
df.Type = df.Type.astype('category')

In [10]:
# As per EDA NaN in data is minimal so dropping NaN

In [11]:
df = df.dropna()

In [12]:
df.isnull().sum()

id                0
Age               0
Gender            0
Type              0
Blood_pressure    0
Cholesterol       0
Heartbeat         0
Thalassemia       0
Result            0
dtype: int64

In [13]:
heart = pd.get_dummies(df)

In [14]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288 entries, 0 to 299
Columns: 304 entries, Age to Type_Typical angina
dtypes: float64(1), int64(5), uint8(298)
memory usage: 99.6 KB


In [15]:
heart.columns

Index(['Age', 'Blood_pressure', 'Cholesterol', 'Heartbeat', 'Thalassemia',
       'Result', 'id_Patient_01', 'id_Patient_02', 'id_Patient_03',
       'id_Patient_04',
       ...
       'id_Patient_96', 'id_Patient_97', 'id_Patient_98', 'id_Patient_99',
       'Gender_Female', 'Gender_Male', 'Type_Asymptomatic',
       'Type_Atypical angina', 'Type_Non-anginal pain', 'Type_Typical angina'],
      dtype='object', length=304)

In [16]:
# The data is now prepared for modeling , 
# As this is a classification task (Not sure what the result column means but that is something we are going to predict here) 
# Doing Basic Random Forest to do classification

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train , X_test , y_train , y_test = train_test_split( heart.drop('Result' , axis = 1) , heart['Result'] )

In [20]:
for i in (X_train , y_train , X_test , y_test):
    print(f'shape : {i.shape}')

shape : (216, 303)
shape : (216,)
shape : (72, 303)
shape : (72,)


In [21]:
from sklearn.metrics import classification_report as cr , confusion_matrix as cm

In [22]:
from sklearn.model_selection import GridSearchCV

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
'''
This is manual code to find n_estimator , 
It is computationally expensive and has possibility of data leakage , So going with GridSearchCV

for n_estimator in range(10 , 1000):
    clf = RandomForestClassifier(n_estimators=n_estimator)
    clf.fit(X_train , y_train)
    y_pred = clf.predict(X_test)
    print(f'For n_estimator value {n_estimator} , The Accuracy score is : {accuracy_score(y_test , y_pred)}')'''

"\nThis is manual code to find n_estimator , \nIt is computationally expensive and has possibility of data leakage , So going with GridSearchCV\n\nfor n_estimator in range(10 , 1000):\n    clf = RandomForestClassifier(n_estimators=n_estimator)\n    clf.fit(X_train , y_train)\n    y_pred = clf.predict(X_test)\n    print(f'For n_estimator value {n_estimator} , The Accuracy score is : {accuracy_score(y_test , y_pred)}')"

In [25]:
# Hyper Parameter Tuning

parameters = {'n_estimators':[10 , 100 , 500 , 1000] , 'criterion':('gini', 'entropy')}


In [26]:
rf_classifier = RandomForestClassifier()

In [27]:
classififier_with_hyper_parameter_tuning = GridSearchCV(rf_classifier , parameters)

In [28]:
import numpy as np

In [29]:
classififier_with_hyper_parameter_tuning.fit( X_train , y_train )

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ('gini', 'entropy'),
                         'n_estimators': [10, 100, 500, 1000]})

In [30]:
classififier_with_hyper_parameter_tuning.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_criterion', 'param_n_estimators', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [31]:
classififier_with_hyper_parameter_tuning.cv_results_

{'mean_fit_time': array([0.02785997, 0.20821657, 1.0122612 , 2.03080025, 0.02728815,
        0.22835193, 1.03441606, 2.14905896]),
 'std_fit_time': array([0.00141521, 0.00514313, 0.0166905 , 0.02669935, 0.00094661,
        0.01568151, 0.02147424, 0.17855559]),
 'mean_score_time': array([0.00592799, 0.01367021, 0.04720602, 0.09183168, 0.00579233,
        0.01478643, 0.04663568, 0.09224143]),
 'std_score_time': array([2.23014827e-04, 6.59588022e-04, 1.62453930e-03, 2.74151247e-03,
        6.95403526e-05, 1.71529920e-03, 3.47820314e-04, 3.34481095e-03]),
 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'entropy', 'entropy',
                    'entropy', 'entropy'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 100, 500, 1000, 10, 100, 500, 1000],
              mask=[False, False, False, False, False, False, False, False],
        fill_valu

In [32]:
classififier_with_hyper_parameter_tuning.best_params_

{'criterion': 'gini', 'n_estimators': 500}

In [33]:
# So We have best score when using criterion as gini and number of estimators as 1000 

In [34]:
actual_model = RandomForestClassifier(criterion='gini' , n_estimators = 100)

In [35]:
actual_model.fit(X_train , y_train)

RandomForestClassifier()

In [36]:
y_pred = actual_model.predict(X_test)

In [37]:
accuracy_score(y_true = y_test , y_pred = y_pred )

0.7916666666666666

In [38]:
# Great we have 81% accurate model , Let's check other criterias

In [39]:
cm( y_test , y_pred )

array([[27,  7],
       [ 8, 30]])

In [40]:
print(cr( y_test , y_pred ))

              precision    recall  f1-score   support

           0       0.77      0.79      0.78        34
           1       0.81      0.79      0.80        38

    accuracy                           0.79        72
   macro avg       0.79      0.79      0.79        72
weighted avg       0.79      0.79      0.79        72



In [41]:
# Based on Confusion Matrix and Classification Report 
# we have a good baseline model with 81% accuracy before jumping deep into ML to find better models

In [42]:
# Thanks